<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Pratical-Keras-Tutorials/Save_and_load_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Dataset

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


# Data Preprocessing

In [3]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

In [4]:
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# Modeling

In [5]:
def create_model():
  model = tf.keras.Sequential([
    # keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.Input(shape=(784,)),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

In [6]:
model = create_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

# Saving the Model

In [7]:
import os

checkpoint_dir = "training_1"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "my_checkpoint.weights.h5")

In [8]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True)

In [9]:
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[checkpoint_cb])

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - loss: 1.6441 - sparse_categorical_accuracy: 0.4700 - val_loss: 0.7207 - val_sparse_categorical_accuracy: 0.8080
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4912 - sparse_categorical_accuracy: 0.8473 - val_loss: 0.5240 - val_sparse_categorical_accuracy: 0.8390
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2987 - sparse_categorical_accuracy: 0.9203 - val_loss: 0.4795 - val_sparse_categorical_accuracy: 0.8530
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2084 - sparse_categorical_accuracy: 0.9422 - val_loss: 0.4420 - val_sparse_categorical_accuracy: 0.8580
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1501 - sparse_categorical_accuracy: 0.9686 - val_loss: 0.4297 - val_sparse_categorical_accuracy: 0.8660
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1164 - sparse_categorical_accuracy: 0.9794 - val_loss: 0.4061 - val_sparse_categorical_accuracy: 0.8730
Epoch 7/10
32/3

In [10]:
os.listdir(checkpoint_dir)

['my_checkpoint.weights.h5']

# Loading the Weights

In [11]:
model = create_model()

In [12]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Untrained model, accuracy: {100 * acc:5.2f}%")

32/32 - 1s - 26ms/step - loss: 2.3874 - sparse_categorical_accuracy: 0.1040
Untrained model, accuracy: 10.40%


In [13]:
model.load_weights("training_1/my_checkpoint.weights.h5")
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"The trained model, accuracy: {100 * acc:5.2f}%")

32/32 - 0s - 3ms/step - loss: 0.4516 - sparse_categorical_accuracy: 0.8620
The trained model, accuracy: 86.20%


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Checkpoint callback options

In [14]:
checkpoint_filepath = "training_2/best.weights.h5"
os.makedirs(os.path.dirname(checkpoint_filepath), exist_ok=True)

batch_size = 32

In [15]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    verbose=1
)

In [16]:
model = create_model()

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(test_images, test_labels),
          callbacks=[checkpoint_cb],
          verbose=0)

/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/model_checkpoint.py:302: UserWarning: Can save best model only with val_accuracy available.
  if self._should_save_model(epoch, batch, logs, filepath):


In [17]:
os.listdir(checkpoint_dir)

['my_checkpoint.weights.h5']

In [18]:
model = create_model()
model.load_weights("training_2/best.weights.h5")

In [19]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Restored model, accuracy: {100 * acc:5.2f}%")

32/32 - 1s - 38ms/step - loss: 0.4840 - sparse_categorical_accuracy: 0.8830
Restored model, accuracy: 88.30%


# EarlyStopping Callback

In [20]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [21]:
checkpoint_path = "training_3/best.weights.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

In [22]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True)

history = model.fit(
    train_images,
    train_labels,
    epochs=50,
    validation_data=(test_images, test_labels),
    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0027 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5054 - val_sparse_categorical_accuracy: 0.8920
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0027 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.6274 - val_sparse_categorical_accuracy: 0.8680
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0027 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5294 - val_sparse_categorical_accuracy: 0.8750
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0013 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5761 - val_sparse_categorical_accuracy: 0.8740
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0012 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5514 - val_sparse_categorical_accuracy: 0.8770
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.5806e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5330 - val_sparse_categorical_accuracy: 0.8870


# Save the entire model

In [23]:
!mkdir saved_model

In [24]:
model.save('saved_model/my_model.keras')

In [25]:
!ls saved_model/my_model.keras

saved_model/my_model.keras


In [26]:
new_model = create_model()
new_model = tf.keras.models.load_model('saved_model/my_model.keras')

In [27]:
new_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

In [28]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print(f'Restored model, accuracy: {100 * acc:5.2f}%')

32/32 - 1s - 25ms/step - loss: 0.5054 - sparse_categorical_accuracy: 0.8920
Restored model, accuracy: 89.20%


Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/evrenozkip) | [Instagram](https://www.instagram.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎